In [1]:
import pandas as pd
import numpy as np
import cv2
import os
from sklearn.utils import shuffle

# 1. Konfigurasi Folder
train_path = 'dataset/train' 
emotion_labels = ['angry', 'disgust', 'fear', 'happy', 'sad', 'surprise', 'neutral']

data_list = []
summary = {label: 0 for label in emotion_labels} # Menghitung statistik tiap emosi
failed_files = []

# --- Inisialisasi CLAHE ---
# Memperjelas fitur wajah (mata/mulut) tanpa menciptakan noise berlebih
clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8,8))

print("--- Tahap 1: Membangun Database (Balanced Contrast) ---")

# 2. Iterasi Folder Emosi
for label_idx, emotion in enumerate(emotion_labels):
    folder_path = os.path.join(train_path, emotion)
    
    if not os.path.exists(folder_path):
        print(f"Peringatan: Folder '{emotion}' tidak ditemukan, melewati...")
        continue
    
    print(f"Memproses kategori: {emotion}...")
    
    for img_name in os.listdir(folder_path):
        if img_name.lower().endswith(('.jpg', '.jpeg', '.png')):
            img_path = os.path.join(folder_path, img_name)
            
            try:
                # Membaca gambar dalam mode Grayscale
                img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
                
                if img is None:
                    continue

                # --- CLAHE (Adaptive Equalization) ---
                img = clahe.apply(img)
                
                # Resize ke standar model 48x48 piksel
                img = cv2.resize(img, (48, 48))
                
                # Mengonversi matriks menjadi string piksel (flatten)
                pixel_string = " ".join(img.flatten().astype(str))
                
                data_list.append([label_idx, pixel_string])
                summary[emotion] += 1
                
            except Exception as e:
                failed_files.append(f"{img_name}: {str(e)}")

# 3. Konversi ke DataFrame & SHUFFLE
# Mengacak urutan data agar distribusi emosi merata di dalam CSV
df = pd.DataFrame(data_list, columns=['emotion', 'pixels'])
df = shuffle(df).reset_index(drop=True) 

# 4. Simpan ke CSV
df.to_csv('database_emosi.csv', index=False)

# 5. Laporan Statistik
print("\n" + "="*40)
print("STATISTIK DATABASE BERHASIL DIBUAT")
print("="*40)
for emosi, jumlah in summary.items():
    print(f"{emosi.ljust(15)}: {jumlah} gambar")

print("-" * 40)
print(f"Total Data Berhasil : {len(df)}")
print(f"Total Data Gagal    : {len(failed_files)}")
print("="*40)

if len(df) > 0:
    print("SUKSES! File 'database_emosi.csv' siap digunakan.")
    print("Silakan lanjut ke Cell 2 (Training).")
else:
    print("EROR: Tidak ada data yang berhasil dikonversi.")

--- Tahap 1: Membangun Database (Balanced Contrast) ---
Memproses kategori: angry...
Memproses kategori: disgust...
Memproses kategori: fear...
Memproses kategori: happy...
Memproses kategori: sad...
Memproses kategori: surprise...
Memproses kategori: neutral...

STATISTIK DATABASE BERHASIL DIBUAT
angry          : 3995 gambar
disgust        : 436 gambar
fear           : 4097 gambar
happy          : 7215 gambar
sad            : 4830 gambar
surprise       : 3171 gambar
neutral        : 4965 gambar
----------------------------------------
Total Data Berhasil : 28709
Total Data Gagal    : 0
SUKSES! File 'database_emosi.csv' siap digunakan.
Silakan lanjut ke Cell 2 (Training).


In [1]:
import pandas as pd
import numpy as np
import os
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Flatten, Dropout, Input, BatchNormalization
from tensorflow.keras.optimizers import Adam
from sklearn.utils import shuffle, class_weight

# 1. Load Data dari Database CSV
print("--- Tahap 1: Memuat Pengetahuan (Database) ---")
if os.path.exists('database_emosi.csv'):
    df = pd.read_csv('database_emosi.csv')
    
    # MENGACAK DATA agar tidak kaku
    df = shuffle(df).reset_index(drop=True)
    print(f"Berhasil memuat {len(df)} data pengalaman.")
    
    print("Distribusi Emosi dalam Database:")
    print(df['emotion'].value_counts().sort_index())
    # Index: 0:Angry, 1:Disgust, 2:Fear, 3:Happy, 4:Sad, 5:Surprise, 6:Neutral
else:
    print("Error: File 'database_emosi.csv' tidak ditemukan! Silakan jalankan Cell 1 dulu.")

# 2. Preprocessing & Normalisasi
print("\n--- Tahap 2: Preprocessing ---")
X = np.array([np.fromstring(p, sep=' ').reshape(48, 48, 1) for p in df['pixels']]) / 255.0
y = df['emotion'].values

# TEKNIK FOKUS: Menyeimbangkan bobot sekaligus memberi "Booster" pada Senang & Terkejut
class_weights = class_weight.compute_class_weight(
    class_weight='balanced',
    classes=np.unique(y),
    y=y
)
class_weights_dict = dict(enumerate(class_weights))

# --- MODIFIKASI MANUAL UNTUK FOKUS ---
class_weights_dict[3] = class_weights_dict[3] * 2.5  # Booster Senang (Happy)
class_weights_dict[5] = class_weights_dict[5] * 2.0  # Booster Terkejut (Surprise)
class_weights_dict[4] = class_weights_dict[4] * 1.5  # Sedikit Booster Sedih (Sad)
class_weights_dict[6] = class_weights_dict[6] * 1.0  # Netral (Standard)

print("Bobot emosi telah dimodifikasi.")

# 3. Arsitektur Model (Ultra-Deep ANN)
print("\n--- Tahap 3: Membangun Otak Buatan (Deep Architecture) ---")
model = Sequential([
    Input(shape=(48, 48, 1)),
    Flatten(), 
    
    # Layer 1: Menangkap pola dasar piksel
    Dense(2048, activation='relu'),
    BatchNormalization(),
    Dropout(0.4),
    
    # Layer 2: Fokus pada fitur wajah
    Dense(1024, activation='relu'),
    BatchNormalization(),
    Dropout(0.3),
    
    # Layer 3: Klasifikasi Emosi
    Dense(512, activation='relu'),
    BatchNormalization(),
    Dropout(0.2),
    
    # Layer 4: Fine-tuning detail
    Dense(256, activation='relu'),
    BatchNormalization(),
    
    # Output Layer: 7 Emosi
    Dense(7, activation='softmax') 
])

# Menggunakan Adam dengan learning rate yang dinamis
optimizer = Adam(learning_rate=0.0005)

model.compile(
    optimizer=optimizer, 
    loss='sparse_categorical_crossentropy', 
    metrics=['accuracy']
)
print("Arsitektur JST Berhasil dibentuk.")
model.summary()

# 4. Proses Training
print("\n--- Tahap 4: Proses Belajar (Deep Learning) ---")

# Early Stopping: Memantau val_accuracy agar tidak berhenti terlalu cepat
early_stop = tf.keras.callbacks.EarlyStopping(
    monitor='val_accuracy', 
    patience=25, 
    restore_best_weights=True,
    verbose=1
)

# ReduceLR: Jika mentok, perkecil langkah belajar agar lebih teliti
reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(
    monitor='val_loss', 
    factor=0.5, 
    patience=7, 
    min_lr=0.00001,
    verbose=1
)

history = model.fit(
    X, y, 
    epochs=150, 
    batch_size=64, # Batch size diperkecil agar belajar lebih detail per gambar
    validation_split=0.2, 
    class_weight=class_weights_dict, 
    callbacks=[early_stop, reduce_lr],
    verbose=1
)

# 5. Simpan Hasil Belajar
model.save('model_cerdas.keras')

# 6. Ringkasan Hasil
final_acc = history.history['accuracy'][-1] * 100
final_val_acc = history.history['val_accuracy'][-1] * 100

print("\n" + "="*40)
print("RINGKASAN HASIL TRAINING")
print("="*40)
print(f"Akurasi Latihan   : {final_acc:.2f}%")
print(f"Akurasi Validasi  : {final_val_acc:.2f}%")
print("-"*40)
print("HASIL ANALISIS:")
if final_val_acc > 40:
    print("SISTEM BERHASIL.")
else:
    print("Sistem masih butuh waktu belajar lebih lama.")
print("="*40)
print("[SUKSES] Model final disimpan sebagai 'model_cerdas.keras'")

--- Tahap 1: Memuat Pengetahuan (Database) ---
Berhasil memuat 28709 data pengalaman.
Distribusi Emosi dalam Database:
emotion
0    3995
1     436
2    4097
3    7215
4    4830
5    3171
6    4965
Name: count, dtype: int64

--- Tahap 2: Preprocessing ---
Bobot emosi telah dimodifikasi.

--- Tahap 3: Membangun Otak Buatan (Deep Architecture) ---
Arsitektur JST Berhasil dibentuk.


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ flatten (Flatten)               │ (None, 2304)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 2048)           │     4,720,640 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 2048)           │         8,192 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 2048)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1024)           │     2,098,176 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_1           │ (None, 1024)           │         4,096 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 1024)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 512)            │       524,800 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_2           │ (None, 512)            │         2,048 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 512)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 256)            │       131,328 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_3           │ (None, 256)            │         1,024 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 7)              │         1,799 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 7,492,103 (28.58 MB)

 Trainable params: 7,484,423 (28.55 MB)

 Non-trainable params: 7,680 (30.00 KB)


--- Tahap 4: Proses Belajar (Deep Learning) ---
Epoch 1/150
359/359 ━━━━━━━━━━━━━━━━━━━━ 39s 95ms/step - accuracy: 0.2650 - loss: 2.7360 - val_accuracy: 0.3332 - val_loss: 1.7411 - learning_rate: 5.0000e-04
Epoch 2/150
359/359 ━━━━━━━━━━━━━━━━━━━━ 38s 85ms/step - accuracy: 0.3274 - loss: 2.3151 - val_accuracy: 0.3400 - val_loss: 1.7656 - learning_rate: 5.0000e-04
Epoch 3/150
359/359 ━━━━━━━━━━━━━━━━━━━━ 30s 85ms/step - accuracy: 0.3533 - loss: 2.1951 - val_accuracy: 0.3187 - val_loss: 1.7190 - learning_rate: 5.0000e-04
Epoch 4/150
359/359 ━━━━━━━━━━━━━━━━━━━━ 30s 84ms/step - accuracy: 0.3654 - loss: 2.0991 - val_accuracy: 0.3386 - val_loss: 1.7212 - learning_rate: 5.0000e-04
Epoch 5/150
359/359 ━━━━━━━━━━━━━━━━━━━━ 30s 83ms/step - accuracy: 0.3770 - loss: 2.0525 - val_accuracy: 0.3339 - val_loss: 1.7467 - learning_rate: 5.0000e-04
Epoch 6/150
359/359 ━━━━━━━━━━━━━━━━━━━━ 30s 83ms/step - accuracy: 0.3935 - loss: 1.9801 - val_accuracy: 0.2682 - val_loss: 1.9970 - learning_rate: 5.0000e-

In [1]:
import cv2
import numpy as np
from tensorflow.keras.models import load_model
from collections import deque

# 1. Konfigurasi Awal
model_path = 'model_cerdas.keras'
label_emosi = ['Marah', 'Jijik', 'Takut', 'Senang', 'Sedih', 'Terkejut', 'Netral']

# Warna khusus untuk setiap emosi (BGR Format)
colors = {
    'Marah': (0, 0, 255),      # Merah
    'Jijik': (0, 255, 128),    # Hijau Muda
    'Takut': (255, 0, 255),    # Ungu
    'Senang': (0, 255, 255),   # Kuning
    'Sedih': (255, 0, 0),      # Biru
    'Terkejut': (255, 255, 0), # Cyan
    'Netral': (200, 200, 200)  # Abu-abu
}

# FITUR STABILISATOR: Menyimpan history agar transisi emosi halus (Anti-Loncat)
history_size = 20
prediksi_history = deque(maxlen=history_size)

# Inisialisasi CLAHE agar hasil kamera sinkron dengan preprocessing di Cell 1 & 2
clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8,8))

print("--- Tahap Akhir: Menyalakan Mata Sistem (Kamera) ---")

# 2. Muat Otak Sistem (Model)
try:
    model = load_model(model_path)
    print(f"[BERHASIL] Memuat model: {model_path}")
except Exception as e:
    print(f"[ERROR] Model gagal dimuat: {e}")

# 3. Muat Detektor Wajah (Haar Cascade)
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

# 4. Inisialisasi Kamera
cap = cv2.VideoCapture(0)

if not cap.isOpened():
    print("[ERROR] Kamera tidak dapat diakses!")
else:
    print("SISTEM AKTIF. Tekan 'q' untuk berhenti.")

while True:
    ret, frame = cap.read()
    if not ret: break

    frame = cv2.flip(frame, 1) # Efek cermin
    gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    
    # Deteksi Wajah dalam frame
    faces = face_cascade.detectMultiScale(gray_frame, 1.3, 5)

    for (x, y, w, h) in faces:
        # A. PREPROCESSING (Wajib sama dengan Cell 1 & 2)
        roi_gray = gray_frame[y:y+h, x:x+w]
        
        # Menerapkan CLAHE (Adaptive Histogram Equalization)
        roi_clahe = clahe.apply(roi_gray)
        
        # Resize ke 48x48 sesuai input model ANN
        roi_resized = cv2.resize(roi_clahe, (48, 48))
        roi_normalized = roi_resized / 255.0
        roi_reshaped = np.reshape(roi_normalized, (1, 48, 48, 1))
        
        # B. PREDIKSI MENTAH
        raw_prediction = model.predict(roi_reshaped, verbose=0)[0]
        
        # C. PROSES SMOOTHING (Menghitung rata-rata prediksi terakhir)
        prediksi_history.append(raw_prediction)
        smoothed_prediction = np.mean(prediksi_history, axis=0)
        
        max_index = np.argmax(smoothed_prediction)
        hasil_prediksi = label_emosi[max_index]
        akurasi = smoothed_prediction[max_index] * 100
        
        # Ambil warna sesuai label emosi
        color = colors.get(hasil_prediksi, (0, 255, 0))

        # D. TAMPILAN HUD (CYBERPUNK STYLE)
        # 1. Gambar Sudut Kotak (Corner Borders)
        length = int(w * 0.2)
        cv2.rectangle(frame, (x, y), (x+w, y+h), color, 1) # Garis tipis kotak luar
        
        # Sudut Kiri Atas
        cv2.line(frame, (x, y), (x+length, y), color, 4)
        cv2.line(frame, (x, y), (x, y+length), color, 4)
        # Sudut Kanan Atas
        cv2.line(frame, (x+w, y), (x+w-length, y), color, 4)
        cv2.line(frame, (x+w, y), (x+w, y+length), color, 4)
        # Sudut Kiri Bawah
        cv2.line(frame, (x, y+h), (x+length, y+h), color, 4)
        cv2.line(frame, (x, y+h), (x, y+h-length), color, 4)
        # Sudut Kanan Bawah
        cv2.line(frame, (x+w, y+h), (x+w-length, y+h), color, 4)
        cv2.line(frame, (x+w, y+h), (x+w, y+h-length), color, 4)
        
        # 2. Label Banner Atas (Latar belakang teks)
        cv2.rectangle(frame, (x, y-40), (x+w, y), color, -1)
        teks = f"{hasil_prediksi.upper()} {akurasi:.1f}%"
        cv2.putText(frame, teks, (x + 5, y - 10), 
                    cv2.FONT_HERSHEY_DUPLEX, 0.7, (255, 255, 255), 1)

    # UI Informasi Header di pojok kiri atas
    cv2.putText(frame, "AI EMOTION ENGINE", (20, 40), 
                cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)
    cv2.putText(frame, "PRESS 'Q' TO SHUTDOWN", (20, 65), 
                cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 255), 1)

    # Tampilkan jendela output
    cv2.imshow('Sistem Cerdas Emotion Detection', frame)

    # Berhenti jika tombol 'q' ditekan
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Bersihkan resources
cap.release()
cv2.destroyAllWindows()
print("--- Sistem Dimatikan: Kamera telah dilepaskan ---")

--- Tahap Akhir: Menyalakan Mata Sistem (Kamera) ---
[BERHASIL] Memuat model: model_cerdas.keras
SISTEM AKTIF. Tekan 'q' untuk berhenti.
